In [1]:
import numpy as np
import math
import random
import pandas as pd
import plotly.express as px
import multiprocessing

np.set_printoptions(threshold=np.inf)


def sigmoid(z: np.array):
    return 1 / (1 + np.exp(-np.clip(z, -100, 100)))


def tanh(z: np.array):
    z = np.clip(z, -100, 100)
    expZ = np.exp(z)
    expZ_minus = np.exp(-z)
    return (expZ - expZ_minus) / (expZ + expZ_minus)


def reLU(z: np.array):
    return np.maximum(0.0, z)


def leakyReLU(z: np.array):
    return np.maximum(0.01 * z, z)

In [2]:
class Model:
    class Layer:
        def __init__(self, layerHeight, prevLayer, nextLayer, activationFunc="sigmoid"):
            self.layerHeight = layerHeight
            self.w = np.random.randn(layerHeight, prevLayer.layerHeight) * np.sqrt(1 / prevLayer.layerHeight)
            self.b = np.zeros((layerHeight, 1))
            self.v_dw = np.zeros((layerHeight, prevLayer.layerHeight))
            self.v_db = np.zeros((layerHeight, 1))
            self.s_dw = np.zeros((layerHeight, prevLayer.layerHeight))
            self.s_db = np.zeros((layerHeight, 1))
            self.t = 1
            self.a = None
            self.d = None
            if activationFunc == "sigmoid":
                self.activation = sigmoid
                self.da_dz = lambda a: np.multiply(a, (1 - a))
            elif activationFunc == "tanh":
                self.activation = np.tanh
                self.da_dz = lambda a: (1 - np.square(a))
            elif activationFunc == "relu":
                self.activation = reLU
                self.da_dz = lambda a: np.where(a > 0, 1,
                                                0)  #just taking advantage of the fact a = z for positive and a = 0 for negative
            elif activationFunc == "leaky_relu":
                self.activation = leakyReLU
                self.da_dz = lambda a: np.where(a > 0, 1, 0.01)
            self.prevLayer = prevLayer if prevLayer else None
            self.nextLayer = nextLayer if nextLayer else None

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.d = np.random.random((self.layerHeight, a_Prev.shape[1])) < keepActiveProb
            self.a = self.activation((self.w @ a_Prev) + self.b) * self.d
            self.a /= keepActiveProb
            # self.a = self.activation((self.w @ a_Prev) + self.b)
            return self.nextLayer.predict(self.a, keepActiveProb)

        def propagate(self, dz_Next: np.array, trainingStep: float, l2_lambda=0):
            da = (self.nextLayer.w.T @ dz_Next) * self.d
            dz = self.da_dz(self.a) * da
            dw = ((dz @ self.prevLayer.a.T) / dz.shape[1]) + ((l2_lambda / dz.shape[1]) * self.w)
            db = np.mean(dz, axis=1, keepdims=True)

            beta1 = 0.9
            beta2 = 0.99
            self.v_dw = (beta1 * self.v_dw) + ((1 - beta1) * dw)
            self.v_db = (beta1 * self.v_db) + ((1 - beta1) * db)
            self.s_dw = (beta2 * self.s_dw) + ((1 - beta2) * np.square(dw))
            self.s_db = (beta2 * self.s_db) + ((1 - beta2) * np.square(db))
            v_dw_cor = self.v_dw / (1 - (beta1 ** self.t))
            v_db_cor = self.v_db / (1 - (beta1 ** self.t))
            s_dw_cor = self.s_dw / (1 - (beta2 ** self.t))
            s_db_cor = self.s_db / (1 - (beta2 ** self.t))

            self.prevLayer.propagate(dz, trainingStep, l2_lambda)
            self.w -= trainingStep * (v_dw_cor / (np.sqrt(s_dw_cor) + (10 ** -8)))
            self.b -= trainingStep * (v_db_cor / (np.sqrt(s_db_cor) + (10 ** -8)))
            self.t += 1

    class Head(Layer):
        def __init__(self, layerHeight, prevLayer, nextLayer, activationFunc="sigmoid", costFunc="logistic"):
            super().__init__(layerHeight, prevLayer, nextLayer, activationFunc)
            if costFunc == "logistic":
                self.dy_hat = lambda y_hat, y: ((1 - y) / (1 - y_hat + 10 ** -8)) - (y / (y_hat + 10 ** -8))
                self.error = lambda y_hat, y: -np.mean(
                    (y * np.log(np.clip(y_hat, 0.001, 1))) + (
                            (1 - y) * np.log(np.clip(1 - y_hat, 0.001, 1))))
            if costFunc == "r2":
                self.dy_hat = lambda y_hat, y: 2 * (y_hat - y)
                self.error = lambda y_hat, y: np.mean(np.square(y - y_hat))

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.a = self.activation((self.w @ a_Prev) + self.b)
            return self.a

        def propagate(self, realY, trainingStep, l2_lambda=0):
            da = self.dy_hat(self.a, realY)
            dz = self.da_dz(self.a) * da
            dw = ((dz @ self.prevLayer.a.T) / dz.shape[1]) + ((l2_lambda / dz.shape[1]) * self.w)
            db = np.mean(dz, axis=1, keepdims=True)

            beta1 = 0.9
            beta2 = 0.999
            self.v_dw = (beta1 * self.v_dw) + ((1 - beta1) * dw)
            self.v_db = (beta1 * self.v_db) + ((1 - beta1) * db)
            self.s_dw = (beta2 * self.s_dw) + ((1 - beta2) * np.square(dw))
            self.s_db = (beta2 * self.s_db) + ((1 - beta2) * np.square(db))
            v_dw_cor = self.v_dw / (1 - (beta1 ** self.t))
            v_db_cor = self.v_db / (1 - (beta1 ** self.t))
            s_dw_cor = self.s_dw / (1 - (beta2 ** self.t))
            s_db_cor = self.s_db / (1 - (beta2 ** self.t))

            self.prevLayer.propagate(dz, trainingStep)
            self.w -= trainingStep * (v_dw_cor / (np.sqrt(s_dw_cor) + (10 ** -8)))
            self.b -= trainingStep * (v_db_cor / (np.sqrt(s_db_cor) + (10 ** -8)))
            self.t += 1

    class Input(Layer):
        def __init__(self, layerHeight, nextLayer):
            self.layerHeight = layerHeight
            self.a = None
            self.w = np.identity(self.layerHeight)
            self.b = np.zeros((layerHeight, 1))
            self.nextLayer = nextLayer if nextLayer else None

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.a = (self.w @ a_Prev) + self.b
            return self.nextLayer.predict(self.a, keepActiveProb)

        def propagate(self, realY, trainingStep, l2_lambda=0):
            return

    def __init__(self, xSize, layers, activationFunc="tanh", headActivation="sigmoid", costFunc="logistic"):
        """
        activationFunc = "sigmoid" | "tanh" | "relu" | "leaky_relu" \n
        costFunc = "logistic" | "r2"
        """
        self.head = None

        def createLayer(prevLayer, depth):
            if depth >= len(layers):
                layer = self.Head(1, prevLayer, None, activationFunc=headActivation, costFunc=costFunc)
                self.head = layer
                return layer
            else:
                layer = self.Layer(layers[depth], prevLayer, None, activationFunc=activationFunc)
                layer.nextLayer = createLayer(layer, depth + 1)
                return layer

        self.input = self.Input(xSize, None)
        self.input.nextLayer = createLayer(self.input, 0)
        self.trainingData = pd.DataFrame([], columns=['Name', 'Iteration', 'Value'])

    def predict(self, a: np.array, keepActiveProb=1.0):
        return self.input.predict(a, keepActiveProb)

    def train(self, X: np.array, Y: np.array, trainingStep, trainingIterations,
              debugStep=100, x_dev=None, y_dev=None, epochSize=1024, keepActiveProb=1.0, l2_reg_lambda=0):
        """
        epochSize ~ 128 | 256 | 512 \n
        keepActiveProb ~ ratio of active neurons {>0.8} \n
        l2_reg_lambda ~ weight decay coefficient {<2.0} \n
        """
        trainSetMean = np.mean(X, axis=1, keepdims=True)
        trainSetR2 = np.mean(np.square(X - trainSetMean), axis=1, keepdims=True)
        self.input.w = self.input.w / np.sqrt(trainSetR2)
        self.input.b = -trainSetMean / np.sqrt(trainSetR2)
        epochs = X.shape[1] // epochSize
        display('epochs Amount: ' + str(epochs))
        for tr_i in range(trainingIterations):
            perm_indices = np.random.permutation(X.shape[1])
            x = X[:, perm_indices]
            y = Y[:, perm_indices]
            for e in range(epochs):
                self.predict(x[:, epochSize * e: epochSize * (e + 1)], keepActiveProb)
                self.head.propagate(y[:, epochSize * e: epochSize * (e + 1)], trainingStep, l2_reg_lambda)
            if x_dev is not None and tr_i % debugStep == 0:
                self.predict(x_dev)
                self.trainingData.loc[len(self.trainingData)] = (
                'ErrorDev', tr_i // debugStep, self.head.error(self.head.a, y_dev))
                # self.trainingData.loc[len(self.trainingData)] = (
                #     'PrecisionDev', tr_i // debugStep, np.mean((y_dev == (self.head.a > 0.5)).astype(int)))
            if tr_i % debugStep == 0:
                self.predict(x)
                self.trainingData.loc[len(self.trainingData)] = (
                'Error', tr_i // debugStep, self.head.error(self.head.a, y))
                # self.trainingData.loc[len(self.trainingData)] = (
                #     'Precision', tr_i // debugStep, np.mean((y == (self.head.a > 0.5)).astype(int)))


In [3]:
df = pd.read_csv('Datasets/coffee-prices-historical-data.csv')
df = df.fillna(df.mean())
x = np.array([[i for i in range(0, int(len(df) * 0.8))]])
x_dev = np.array([[i for i in range(int(len(df) * 0.8), len(df))]])
y = df.loc[x.flatten()][' value'].to_numpy().reshape(1, -1)
y_dev = df.loc[x_dev.flatten()][' value'].to_numpy().reshape(1, -1)
display(x.shape)
display(y.shape)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8448\586355787.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


(1, 10050)

(1, 10050)

In [4]:
df = pd.read_csv('Datasets/coffee-prices-historical-data.csv')
df = df.fillna(df.mean())

x_predictive = np.zeros((1500, int(len(df) * 0.8)))
y_predictive = np.zeros((1, int(len(df) * 0.8)))
for i in range(1, int(len(df) * 0.8) + 1):
    # Calculate the number of prices to include in the i-th column
    num_prices_to_include = min(1500, i)

    # Fill the i-th column with prices from the dataset
    x_predictive[:num_prices_to_include, i - 1] = df[i - num_prices_to_include:i][' value'].to_numpy().reshape((num_prices_to_include,))[::-1]
    y_predictive[0, i - 1] = df.loc[i][' value']
display("Generated")


C:\Users\Admin\AppData\Local\Temp\ipykernel_8448\4023190707.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


'Generated'

In [5]:
df = pd.read_csv('Datasets/coffee-prices-historical-data.csv')
df = df.fillna(df.mean())

x_predictive_dev = np.zeros((1500, len(df) - int(len(df) * 0.8)))
y_predictive_dev = np.zeros((1, len(df) - int(len(df) * 0.8)))
for i in range(int(len(df) * 0.8), len(df)):
    # Calculate the number of prices to include in the i-th column
    num_prices_to_include = min(1500, i)

    # Fill the i-th column with prices from the dataset
    x_predictive_dev[:num_prices_to_include, i - int(len(df) * 0.8)] = df[i - num_prices_to_include:i][' value'].to_numpy().reshape((num_prices_to_include,))[::-1]
    y_predictive_dev[0, i - int(len(df) * 0.8)] = df.loc[i][' value']
display("Generated")

C:\Users\Admin\AppData\Local\Temp\ipykernel_8448\3939530910.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


'Generated'

In [9]:
np.random.seed(69)
model0 = Model(1, [20, 20, 20], activationFunc="tanh", headActivation="relu", costFunc="r2")
model0.train(x, y, 0.005, 251, x_dev=x_dev, y_dev=y_dev, epochSize=256, debugStep=10, l2_reg_lambda=0)

'epochs Amount: 39'

In [78]:
np.random.seed(69)
model_predictive = Model(1500, [5, 5], activationFunc="tanh", headActivation="relu", costFunc="r2")
model_predictive.train(x_predictive, y_predictive, 0.0001, 1001, x_dev=x_predictive_dev, y_dev=y_predictive_dev, epochSize=256, debugStep=1, l2_reg_lambda=0.25)

'epochs Amount: 39'

In [10]:
px.line(model0.trainingData, x='Iteration', y='Value', color='Name')

In [79]:
px.line(model_predictive.trainingData, x='Iteration', y='Value', color='Name')

In [11]:
predict = px.line(x=df.loc[x.flatten()]['date'], y=model0.predict(x).flatten()).update_traces(line=dict(color='white'))
forecast = px.line(x=df.loc[x_dev.flatten()]['date'], y=model0.predict(x_dev).flatten()).update_traces(
    line=dict(color='red'))
data = px.line(x=df['date'], y=df[' value']).update_traces(line=dict(color='green'))
combined_fig = predict.add_traces(data.data)
combined_fig = combined_fig.add_traces(forecast.data)

# Display the combined plot
combined_fig.show()

In [80]:
predict = px.line(x=df.loc[range(1, x_predictive.shape[1] + 1)]['date'], y=model_predictive.predict(x_predictive).flatten()).update_traces(line=dict(color='white'))
forecast = px.line(x=df.loc[range(x_predictive.shape[1] + 1, x_predictive.shape[1] + x_predictive_dev.shape[1])]['date'], y=model_predictive.predict(x_predictive_dev[:,:-1]).flatten()).update_traces(
    line=dict(color='red'))
data = px.line(x=df['date'], y=df[' value']).update_traces(line=dict(color='green'))
combined_fig = predict.add_traces(data.data)
combined_fig = combined_fig.add_traces(forecast.data)

# Display the combined plot
combined_fig.show()

In [ ]:
model_predictive.predict(x_predictive_dev[:,:-1])